In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/mercari'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/mercari/test.tsv
/kaggle/input/mercari/train.tsv
/kaggle/input/mercari/test_stg2.tsv


In [2]:
#importing train data
raw_data=pd.read_csv("../input/mercari/train.tsv",sep="\t")

test_data=pd.read_csv("../input/mercari/test_stg2.tsv",sep="\t")

In [3]:
#converting all text to lower case
raw_data['name']=raw_data['name'].str.lower()
test_data['name']=test_data['name'].str.lower()


In [4]:
## converting all category names to lower case
raw_data['category_name']=raw_data['category_name'].str.lower()
test_data['category_name']=test_data['category_name'].str.lower()

In [5]:
#splitting category into three columns
# https://www.geeksforgeeks.org/python-pandas-split-strings-into-two-list-columns-using-str-split/
sub_category_columns=raw_data['category_name'].str.split("/",expand=True,n=2)
sub_category_columns_test=test_data['category_name'].str.split("/",expand=True,n=2)

In [6]:
raw_data['category']=sub_category_columns[0]
raw_data['subcategory1']=sub_category_columns[1]
raw_data['subcategory2']=sub_category_columns[2]

test_data['category']=sub_category_columns_test[0]
test_data['subcategory1']=sub_category_columns_test[1]
test_data['subcategory2']=sub_category_columns_test[2]

In [7]:
#converting brand names to lower case
raw_data['brand_name']=raw_data['brand_name'].str.lower()
test_data['brand_name']=test_data['brand_name'].str.lower()

In [8]:
# trying to impute brand name
list_of_brand_names=raw_data['brand_name'].unique().tolist()
print(len(list_of_brand_names))
cleaned_brand_list = [x for x in list_of_brand_names if str(x) != 'nan']
print(len(cleaned_brand_list))

4810
4809


In [9]:
def brand_replace(column):
    for i in cleaned_brand_list:
        if i in str(column).split():
            return i

In [10]:
raw_data['Newbrand']=raw_data['name'].apply(brand_replace)

In [11]:
raw_data['brand_name']=np.where(raw_data['brand_name'].isnull(),raw_data['Newbrand'],raw_data['brand_name'])

In [12]:
raw_data['brand_name'].fillna(value="Not Present", inplace=True)
test_data['brand_name'].fillna(value="Not Present", inplace=True)


In [13]:
# converting all item descritpion to lower case
raw_data['item_description']=raw_data['item_description'].str.lower()
test_data['item_description']=test_data['item_description'].str.lower()

In [14]:
raw_data_zero_price_removed=raw_data[raw_data.price!=0]

In [15]:
raw_data_zero_price_removed['item_description'].fillna(value='Not Present',inplace=True)
test_data['item_description'].fillna(value='Not Present',inplace=True)

/opt/conda/lib/python3.6/site-packages/pandas/core/generic.py:6287: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [16]:
raw_data_zero_price_removed['category'].fillna(value='Not Present',inplace=True)
raw_data_zero_price_removed['subcategory1'].fillna(value='Not Present',inplace=True)
raw_data_zero_price_removed['subcategory2'].fillna(value='Not Present',inplace=True)


test_data['category'].fillna(value='Not Present',inplace=True)
test_data['subcategory1'].fillna(value='Not Present',inplace=True)
test_data['subcategory2'].fillna(value='Not Present',inplace=True)

In [17]:
raw_data_zero_price_removed['log_price']=np.log(raw_data_zero_price_removed['price']+1)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [18]:
y=raw_data_zero_price_removed.log_price
x=raw_data_zero_price_removed.drop(['train_id','item_condition_id','category_name','Newbrand','log_price'],axis=1)

In [19]:
def decontracted(phrase):
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [20]:
from nltk.corpus import stopwords
stopwords=stopwords.words('english')
import re

In [21]:
len(stopwords)


from tqdm import tqdm
preprocessed_name_train = []
for sentance in tqdm(x['name'].values):
    sent = decontracted(sentance)
    sent = sent.replace('\\r', ' ')
    sent = sent.replace('\\"', ' ')
    sent = sent.replace('\\n', ' ')
    sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
    
    sent = ' '.join(e for e in sent.split() if e not in stopwords)
    preprocessed_name_train.append(sent.lower().strip())

preprocessed_description_train = []
for sentance in tqdm(x['item_description'].values):
    sent = decontracted(sentance)
    sent = sent.replace('\\r', ' ')
    sent = sent.replace('\\"', ' ')
    sent = sent.replace('\\n', ' ')
    sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
    
    sent = ' '.join(e for e in sent.split() if e not in stopwords)
    preprocessed_description_train.append(sent.lower().strip())
    
preprocessed_name_test = []
for sentance in tqdm(test_data['name'].values):
    sent = decontracted(sentance)
    sent = sent.replace('\\r', ' ')
    sent = sent.replace('\\"', ' ')
    sent = sent.replace('\\n', ' ')
    sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
    
    sent = ' '.join(e for e in sent.split() if e not in stopwords)
    preprocessed_name_test.append(sent.lower().strip())
    
preprocessed_description_test = []
for sentance in tqdm(test_data['item_description'].values):
    sent = decontracted(sentance)
    sent = sent.replace('\\r', ' ')
    sent = sent.replace('\\"', ' ')
    sent = sent.replace('\\n', ' ')
    sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
    
    sent = ' '.join(e for e in sent.split() if e not in stopwords)
    preprocessed_description_test.append(sent.lower().strip())
    
preprocessed_brand_name_train = []
for sentance in tqdm(x['brand_name'].values):
    sent = decontracted(sentance)
    sent = sent.replace('\\r', ' ')
    sent = sent.replace('\\"', ' ')
    sent = sent.replace('\\n', ' ')
    sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
    
    sent = ' '.join(e for e in sent.split() if e not in stopwords)
    preprocessed_brand_name_train.append(sent.lower().strip())
    
preprocessed_brand_name_test = []
for sentance in tqdm(test_data['brand_name'].values):
    sent = decontracted(sentance)
    sent = sent.replace('\\r', ' ')
    sent = sent.replace('\\"', ' ')
    sent = sent.replace('\\n', ' ')
    sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
    
    sent = ' '.join(e for e in sent.split() if e not in stopwords)
    preprocessed_brand_name_test.append(sent.lower().strip())

100%|██████████| 3460725/3460725 [01:44<00:00, 32989.54it/s]


In [22]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [23]:
count_name = CountVectorizer(min_df=10)
train_name_vec = count_name.fit_transform(x["name"])
test_name_vec = count_name.transform(test_data["name"])
print("Shape of train Name feature: ",train_name_vec.shape)
print("Shape of test Name feature: ",test_name_vec.shape)

Shape of train Name feature:  (1481661, 17732)
Shape of test Name feature:  (3460725, 17732)


In [24]:
## TFIDF on description
vectorizer_desc_tfidf = TfidfVectorizer(min_df=10,max_features=50000,ngram_range=(1,3))
desc_tfidf_train = vectorizer_desc_tfidf.fit_transform(preprocessed_description_train)

print("Shape of matrix after TFIDF vectorizer ",desc_tfidf_train.shape)

## TFIDF on description test set
desc_tfidf_test = vectorizer_desc_tfidf.transform(preprocessed_description_test)

print("Shape of matrix after TFIDF vectorizer ",desc_tfidf_test.shape)

Shape of matrix after TFIDF vectorizer  (1481661, 50000)
Shape of matrix after TFIDF vectorizer  (3460725, 50000)


In [25]:
# featurizing brand name
from sklearn.preprocessing import LabelBinarizer

print("Brand encoders")
vect_brand = LabelBinarizer(sparse_output=True)

encoder_brand_train = vect_brand.fit_transform(x["brand_name"])
encoder_brand_test= vect_brand.transform(test_data["brand_name"])

Brand encoders


In [26]:
import category_encoders as ce
cat_features = ['category', 'subcategory1', 'subcategory2']
count_enc = ce.CountEncoder()
count_encoded_train = count_enc.fit_transform(x[cat_features])
count_encoded_test=count_enc.transform(test_data[cat_features])

In [27]:
x_train_count_encoded=x.join(count_encoded_train.add_suffix("_count"))
x_test_count_encoded=test_data.join(count_encoded_test.add_suffix("_count"))

In [28]:
x_test_count_encoded['category_count'].fillna(0,inplace=True)
x_test_count_encoded['subcategory1_count'].fillna(0,inplace=True)
x_test_count_encoded['subcategory2_count'].fillna(0,inplace=True)

In [29]:
##brand average price
avg_brand_price=pd.DataFrame(x_train_count_encoded.groupby('brand_name',as_index=False)['price'].mean())

avg_brand_price['log_price']=np.log(avg_brand_price['price']+1)

#adding average price of each brand in x_train
x_train_count_encoded=pd.merge(x_train_count_encoded,avg_brand_price[['brand_name','log_price']],how='left',on='brand_name')
x_test_count_encoded=pd.merge(x_test_count_encoded,avg_brand_price[['brand_name','log_price']],how='left',on='brand_name')
x_train_count_encoded.rename(columns={'log_price':'avg_brand_price'}, inplace=True)
x_test_count_encoded.rename(columns={'log_price':'avg_brand_price'},inplace=True)

In [30]:
##category average price
avg_cat_price=pd.DataFrame(x_train_count_encoded.groupby('category',as_index=False)['price'].mean())

avg_cat_price['log_price']=np.log(avg_cat_price['price']+1)

x_train_count_encoded=pd.merge(x_train_count_encoded,avg_cat_price[['category','log_price']],how='left',on='category')

x_test_count_encoded=pd.merge(x_test_count_encoded,avg_cat_price[['category','log_price']],how='left',on='category')

x_train_count_encoded.rename(columns={'log_price':'avg_cat_price'}, inplace=True)
x_test_count_encoded.rename(columns={'log_price':'avg_cat_price'},inplace=True)

In [31]:
##subcategory average price
avg_subcat1_price=pd.DataFrame(x_train_count_encoded.groupby('subcategory1',as_index=False)['price'].mean())

avg_subcat1_price['log_price']=np.log(avg_subcat1_price['price']+1)


##updating average subcategory price in train and test
x_train_count_encoded=pd.merge(x_train_count_encoded,avg_subcat1_price[['subcategory1','log_price']],how='left',on='subcategory1')

x_test_count_encoded=pd.merge(x_test_count_encoded,avg_subcat1_price[['subcategory1','log_price']],how='left',on='subcategory1')

x_train_count_encoded.rename(columns={'log_price':'avg_subcat1_price'}, inplace=True)
x_test_count_encoded.rename(columns={'log_price':'avg_subcat1_price'},inplace=True)

In [32]:
##subcategory average price
avg_subcat2_price=pd.DataFrame(x_train_count_encoded.groupby('subcategory2',as_index=False)['price'].mean())

avg_subcat2_price['log_price']=np.log(avg_subcat2_price['price']+1)


##updating average subcategory price in train and test
x_train_count_encoded=pd.merge(x_train_count_encoded,avg_subcat2_price[['subcategory2','log_price']],how='left',on='subcategory2')

x_test_count_encoded=pd.merge(x_test_count_encoded,avg_subcat2_price[['subcategory2','log_price']],how='left',on='subcategory2')

x_train_count_encoded.rename(columns={'log_price':'avg_subcat2_price'}, inplace=True)
x_test_count_encoded.rename(columns={'log_price':'avg_subcat2_price'},inplace=True)

In [33]:
x_test_count_encoded['avg_brand_price'].fillna(0,inplace=True)
x_test_count_encoded['avg_cat_price'].fillna(0,inplace=True)
x_test_count_encoded['avg_subcat1_price'].fillna(0,inplace=True)
x_test_count_encoded['avg_subcat2_price'].fillna(0,inplace=True)

In [34]:
from scipy.sparse import hstack
# with the same hstack function we are concatinating a sparse matrix and a dense matirx :)
X_TRAIN = hstack((train_name_vec,desc_tfidf_train,encoder_brand_train,x_train_count_encoded[['shipping','category_count','subcategory1_count','subcategory2_count','avg_brand_price','avg_cat_price','avg_subcat1_price','avg_subcat2_price']])).tocsr()
print(X_TRAIN.shape)

X_TEST = hstack((test_name_vec,desc_tfidf_test,encoder_brand_test,x_test_count_encoded[['shipping','category_count','subcategory1_count','subcategory2_count','avg_brand_price','avg_cat_price','avg_subcat1_price','avg_subcat2_price']])).tocsr()
print(X_TEST.shape)

(1481661, 72548)
(3460725, 72548)


In [35]:
y_train=raw_data_zero_price_removed.log_price

In [36]:
import gc
del(count_name)
del(vectorizer_desc_tfidf)
del(vect_brand)
del(count_enc)

gc.collect()

23

In [37]:
del(avg_brand_price)
del(avg_cat_price)
del(avg_subcat1_price)
del(avg_subcat2_price)

gc.collect()

0

In [38]:
del(preprocessed_name_train)
del(preprocessed_name_test)
del(preprocessed_description_train)
del(preprocessed_description_test)
del(preprocessed_brand_name_train)
del(preprocessed_brand_name_test)

In [39]:
gc.collect()

0

In [40]:
from xgboost import XGBRegressor
xgb = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bytree=1, gamma=0.7,learning_rate=0.1, max_delta_step=0,
             max_depth=16, min_child_weight=1, missing=None, n_estimators=200,
             n_jobs=-1, random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1, 
             seed=None, silent=True, subsample=1)

print("Fitting Model 1")
xgb.fit(X_TRAIN, y_train)

Fitting Model 1


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0.7, gpu_id=-1,
             importance_type='gain', interaction_constraints=None,
             learning_rate=0.1, max_delta_step=0, max_depth=16,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             n_estimators=200, n_jobs=-1, num_parallel_tree=1,
             objective='reg:squarederror', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, seed=0, silent=True, subsample=1,
             tree_method=None, validate_parameters=False, verbosity=None)

In [41]:
y_pred_xg = xgb.predict(X_TEST)

In [42]:
y_pred_xg[y_pred_xg<0]=0

In [43]:
y_pred_xg=np.exp(y_pred_xg)-1

In [44]:
test_data['price']=y_pred_xg

In [45]:
test_data.head(2)

,test_id,name,item_condition_id,category_name,brand_name,shipping,item_description,category,subcategory1,subcategory2,price
0,0,"breast cancer ""i fight like a girl"" ring",1,women/jewelry/rings,Not Present,1,size 7,women,jewelry,rings,10.563006
1,1,"25 pcs new 7.5""x12"" kraft bubble mailers",1,other/office supplies/shipping supplies,Not Present,1,"25 pcs new 7.5""x12"" kraft bubble mailers lined...",other,office supplies,shipping supplies,9.368939


In [46]:
sub1=test_data[['test_id','price']]

In [47]:
sub1.to_csv('submission.csv',index=False)